# Meter los datos en la base de datos

Puedes ejecutar este notebook desde la linea de comandos con:

```bash
papermill --cwd ./chat -k python3 --no-log-output chat/add_info_python.ipynb /tmp/output.log.json
```

In [1]:
import os
from tqdm.notebook import tqdm
import more_itertools
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

print(f"Inicializando chromadb")
NUM_CHUNKS = 7
MODEL = "phi3"

chroma_client = chromadb.PersistentClient(path='db') # current folder
embedding_function = embedding_functions.OllamaEmbeddingFunction(
    url="http://localhost:11434/api/embeddings", # ollama running in the background
    model_name=MODEL,
)

collection = chroma_client.get_or_create_collection(name="turras",
                                                      embedding_function=embedding_function)

Inicializando chromadb


# Meter términos de glosario


In [2]:
import csv
print("Glosario loader")
def glosario():
    file_path = os.path.join(os.getcwd(), '..', 'db', 'glosario.csv')
    
    with open(file_path, mode='r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, fieldnames=['word', 'definition', 'sources'], delimiter=',', quotechar='"')
        next(reader) # header
        for row in reader:
            yield {
                'word': row['word'],
                'definition': row['definition'],
                'sources': row['sources'] or ''
            }

glos_documents = []
glos_ids = []

for item in glosario():
    glos_documents.append(f"{item['word']}: {item['definition']}")
    glos_ids.append(item['word'])
    
ids=list(more_itertools.chunked(glos_documents, NUM_CHUNKS))
docs=list(more_itertools.chunked(glos_ids, NUM_CHUNKS))
for ids, docs in tqdm(list(zip(ids, docs))):
    collection.upsert(documents=glos_documents, ids=glos_ids)

Glosario loader


  0%|          | 0/9 [00:00<?, ?it/s]

## Meter las turras

In [3]:
import json
from pprint import pprint
import os
from tqdm import tqdm

print(f"Turra loader")
NUM_CHUNKS = 7

def turras():
    urls = []
    completas = []

    with open(os.path.join(os.getcwd(), '..', 'db', 'tweets.json'), mode='r', encoding='utf-8') as jsonfile:
        turras = json.load(jsonfile)
        for turra in turras:
            try:
                i = str(turra[0]['id'])
                urls.append("https://turrero.vercel.app/turra/" + i)
                completas.append('\n'.join(tuit["tweet"] for tuit in turra))
            except Exception as e:
                pprint(turra)
                raise e
    assert len(urls) == len(completas), "Bug: no hay la misma cantidad de urls y turras"
    return (urls, completas)

# Partirlo en cachos para poder ver la barrita llenarse, pues tarda mucho
urls_list, turras_list = turras()

urls_chunks = list(more_itertools.chunked(urls_list, NUM_CHUNKS))
turras_chunks = list(more_itertools.chunked(turras_list, NUM_CHUNKS))
batched = list(zip(urls_chunks, turras_chunks))

for ids, docs in tqdm(batched):
    collection.upsert(
        documents=docs,
        ids=ids
    )

Turra loader


100%|██████████| 27/27 [1:04:19<00:00, 142.93s/it]


# Ejemplo de Query

In [4]:
turras = collection.query(query_texts=["¿qué es la estrategia?"], n_results=3)
turras

{'ids': [['https://turrero.vercel.app/turra/1733397009418096842',
   'https://turrero.vercel.app/turra/1710564695336247487',
   'https://turrero.vercel.app/turra/1619227404794306560']],
 'distances': [[1306.1426099527098, 1310.3254430705897, 1321.101318359375]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['En el hilo turras de hoy, vamos a hablar de este tema fascinante: niños y CPS. Cojamos la merienda y vamos a ello.\nRepasando un poco para hacer el hilo me he encontrado que ha sido una cosa que ha estado flotando en la comunidad desde siempre.\nPorque fundamentalmente, todos tenemos la intuición de que la formación en CPS es algo tan técnico como humanístico, y si me apuras filosófico. Así que tiene sentido que pensemos en educar a los niños de manera acorde.\nEl CPS para niños tiene dos acepciones. Voy a rápidamente quitarme de en medio la primera.\nLas mecánicas sencillas de descomposición de problemas en elementos que las componen y las sistemáticas s

In [5]:
definiciones = collection.query(query_texts=["¿qué es la estrategia?"], n_results=5)
definiciones

{'ids': [['https://turrero.vercel.app/turra/1733397009418096842',
   'https://turrero.vercel.app/turra/1710564695336247487',
   'https://turrero.vercel.app/turra/1619227404794306560',
   'https://turrero.vercel.app/turra/1157653953716396036',
   'https://turrero.vercel.app/turra/1743547711897342313']],
 'distances': [[1306.1426099527098,
   1310.3254430705897,
   1321.101318359375,
   1322.647216796875,
   1333.1630739304046]],
 'metadatas': [[None, None, None, None, None]],
 'embeddings': None,
 'documents': [['En el hilo turras de hoy, vamos a hablar de este tema fascinante: niños y CPS. Cojamos la merienda y vamos a ello.\nRepasando un poco para hacer el hilo me he encontrado que ha sido una cosa que ha estado flotando en la comunidad desde siempre.\nPorque fundamentalmente, todos tenemos la intuición de que la formación en CPS es algo tan técnico como humanístico, y si me apuras filosófico. Así que tiene sentido que pensemos en educar a los niños de manera acorde.\nEl CPS para niño

In [6]:
import ollama



stream = ollama.chat(
    model=MODEL,
    messages=[{'role': 'user', 'content': '¿qué es la estrategia?'}],
    stream=True,
)


for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

 La estrategia se refiere a un conjunto de acciones planificadas y coordinadas diseñadas para alcanzar un objetivo específico. Es una visión general, que define los pasos necesarios para lograr dicho objetivo, teniendo en cuenta las circunstancias y recursos disponibles. La estrategia puede aplicarse en diversos campos como negocios, deportes, política, educación, entre otros.

Algunas características clave de una buena estrategia incluyen:

1. Claridad: Debería ser fácilmente comprensible por todos involucrados y reflejar la visión general del objetivo a alcanzar.
2. Coordinación: Las acciones dentro de una estrategia deben estar interconectadas, trabajando juntas para cumplir el objetivo final.
3. Flexibilidad: Debe ser adaptable a las circunstancias cambiantes y la entrada de nuevos datos o información.
4. Evaluación y revisión: La estrategia debe incluir mecanismos para evaluar su eficacia, así como revisiones periódicas para hacer ajustes necesarios.
5. Alineamiento con recursos d

In [7]:
prompt = """

Sigue las siguientes instrucciones. Eres un asistente para tareas de tipo 'pregunta y respuesta', solo puedes dar respuestas cortas de máximo un párrafo. Utiliza la información del contexto para crer una respuesta corta, breve y en registro oral, pero no te alargues ni incluyas toda la información del contexto si no es necesario.

Pregunta del usuario: {question} 
Contexto de la conversación: {context} 
Respuesta (en su mismo idioma):
"""

In [8]:
question = '¿qué es la estrategia?'
context = '\n'.join(definiciones['documents'][0] + turras['documents'][0])
full_prompt = prompt.format(question=question, context=context)
len(full_prompt)

full_prompt

'\n\nSigue las siguientes instrucciones. Eres un asistente para tareas de tipo \'pregunta y respuesta\', solo puedes dar respuestas cortas de máximo un párrafo. Utiliza la información del contexto para crer una respuesta corta, breve y en registro oral, pero no te alargues ni incluyas toda la información del contexto si no es necesario.\n\nPregunta del usuario: ¿qué es la estrategia? \nContexto de la conversación: En el hilo turras de hoy, vamos a hablar de este tema fascinante: niños y CPS. Cojamos la merienda y vamos a ello.\nRepasando un poco para hacer el hilo me he encontrado que ha sido una cosa que ha estado flotando en la comunidad desde siempre.\nPorque fundamentalmente, todos tenemos la intuición de que la formación en CPS es algo tan técnico como humanístico, y si me apuras filosófico. Así que tiene sentido que pensemos en educar a los niños de manera acorde.\nEl CPS para niños tiene dos acepciones. Voy a rápidamente quitarme de en medio la primera.\nLas mecánicas sencillas 

In [9]:
stream = ollama.chat(
    model=MODEL,
    messages=[{'role': 'user', 'content': full_prompt}],
    stream=True,
)


for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

 El debate entre "izquierda" y "derecha" es un reflejo complejo de nuestras percepciones y narrativas sobre la sociedad, donde cada bando selecciona y promueve una visión distinta que apoya sus ideales y objetivos. La realidad se complica cuando las diferentes versiones de estas historias chocan en un campo de batalla ideológico lleno de argumentaciones valiosas pero a veces conflictivas.

En el caso del "izquierda", su enfoque suele centrarse en la justicia social, la igualdad y las oportunidades para todos los ciudadanos. A menudo se critica por seleccionar narrativas que apoyen sus propósitos políticos mientras ignoran ciertas realidades históranicas, como las dificultades de implementación de su ideales en la práctica.

Por otro lado, la "derecha" a menudo enfatiza el individualismo y la libertad económica como pilares para un crecimiento sostenible y exitoso. Se critica por no ser suficientemente realista con respecto a las consecuencias de ciertas políticas económicas y sociales,